# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = {'class' : 'SimpleStrategy','replication_factor':1}
""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

## Create queries to ask the following three questions of the data

### 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [8]:
query= "DROP TABLE IF EXISTS song_info"
session.execute(query)
query = "CREATE TABLE IF NOT EXISTS song_info "
query += "(sessionid int,iteminsession int,\
           artist text, song text, length double,\
          PRIMARY KEY (sessionid, iteminsession))"
session.execute(query)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_info (sessionid,iteminsession,artist, song,length) "
        query = query + "VALUES (%s ,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "select artist,song,length,sessionid,iteminsession from song_info WHERE sessionid=338 AND iteminsession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print('artist\t | song\t\t\t\t\t\t\t\t\t\t\t\t\t|length \t|sessionid |iteminsession')
print('-----------------------------------------------------------------------------')
for row in rows:
    print(row.artist, row.song, row.length,row.sessionid,'\t\t\t\t',row.iteminsession)

artist	 | song													|length 	|sessionid |iteminsession
-----------------------------------------------------------------------------
Faithless Music Matters (Mark Knight Dub) 495.3073 338 				 4


### 2. Show only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [11]:
query= "DROP TABLE IF EXISTS song_playlist_session"
session.execute(query)
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query += "(userid int,\
           sessionid int,\
           iteminsession int,\
           artist text, \
           song text,\
           first text,\
           last text,\
           PRIMARY KEY ((userid,sessionid), iteminsession))"
session.execute(query)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userid,sessionid,iteminsession,artist, song,first,last) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [13]:
query = "select artist,song,first,last from song_playlist_session WHERE userid=10 AND sessionid=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print('artist\t\t\t\t | song\t\t\t\t\t\t\t\t\t|name')
print('-----------------------------------------------------------------------------')
for row in rows:
    print(row.artist,'\t',row.song,row.first,row.last)

artist				 | song									|name
-----------------------------------------------------------------------------
Down To The Bone 	 Keep On Keepin' On Sylvie Cruz
Three Drives 	 Greece 2000 Sylvie Cruz
Sebastien Tellier 	 Kilometer Sylvie Cruz
Lonnie Gordon 	 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Show every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
query= "DROP TABLE IF EXISTS user_info"
session.execute(query)
query = "CREATE TABLE IF NOT EXISTS user_info "
query += "(song text,\
           first text,\
           last text,\
           PRIMARY KEY (song,first,last))"
session.execute(query)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_info (song,first,last) "
        query = query + "VALUES (%s ,%s,%s)"
        session.execute(query, (line[9],line[1],line[4]))

In [16]:
query = "select first,last from user_info WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print('user name')
print('--------------------')
for row in rows:
    print(row.first,row.last)

user name
--------------------
Jacqueline Lynch
Sara Johnson
Tegan Levine


### Dropping the tables

In [17]:
query= "DROP TABLE IF EXISTS song_info"
session.execute(query)

query= "DROP TABLE IF EXISTS song_playlist_session"
session.execute(query)

query= "DROP TABLE IF EXISTS user_info"
session.execute(query)

In [18]:
session.shutdown()
cluster.shutdown()